In [6]:
import pandas as pd
import os

# 1. 데이터 로드 함수 (인코딩 자동 감지)
def load_data_smart(file_path):
    encodings = ['utf-8', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            df = pd.read_csv(file_path, encoding=enc)
            print(f"Load Success: {file_path} (Enc: {enc})")
            return df
        except:
            continue
    raise ValueError(f"File Load Failed: {file_path}")

# ========================================================
# 2. 데이터 로드
# ========================================================
file_accident = "수도권 교통사고분석 2021~2024.csv"
file_pop = "수도권 인구 현황 2021~2024.csv"
file_code = "행정구역 코드.csv"  # 새로 바뀐 파일명

df_accident = load_data_smart(file_accident)
df_pop_raw = load_data_smart(file_pop)
df_code = load_data_smart(file_code)

# ========================================================
# 3. 행정구역 코드 정리 (5자리 코드 생성)
# ========================================================
# 시도코드(2자리) + 시군구코드(3자리) = 5자리 표준코드 생성
# 예: 서울(11) + 종로구(10 -> 010) = 11010
df_code['Sido_Str'] = df_code['시도코드'].astype(str).str.zfill(2)
df_code['Sgg_Str'] = df_code['시군구코드'].astype(str).str.zfill(3)
df_code['Region_Code'] = df_code['Sido_Str'] + df_code['Sgg_Str']

# 매핑 키 생성 (예: "서울특별시 종로구")
df_code['Full_Name'] = df_code['시도명칭'] + " " + df_code['시군구명칭']

# 중복 제거 (읍면동 단위까지 있어서 시군구 단위로 중복될 수 있음)
df_code_clean = df_code[['Full_Name', 'Region_Code']].drop_duplicates()

# 매핑 딕셔너리 생성
code_map = df_code_clean.set_index('Full_Name')['Region_Code'].to_dict()

# ========================================================
# 4. 사고 데이터에 적용 및 날짜 변환
# ========================================================
# 사고 데이터의 '시군구'는 '경기도 오산시' 형태 -> 바로 매핑
df_accident['Region_Code'] = df_accident['시군구'].map(code_map)

# 날짜 변환
df_accident['Year'] = df_accident['발생년월'].astype(str).str[:4].astype(int)
df_accident['Month'] = df_accident['발생년월'].astype(str).str.extract(r'(\d+)월')[0].astype(int)

# ========================================================
# 5. 결과 확인
# ========================================================
print("\n--- 매핑 결과 ---")
print(f"매핑 성공: {df_accident['Region_Code'].notna().sum()} / {len(df_accident)}")

missing = df_accident[df_accident['Region_Code'].isna()]['시군구'].unique()
if len(missing) > 0:
    print(f"⚠️ 매핑 실패 지역 ({len(missing)}개): {missing}")
else:
    print("✅ 모든 지역 코드가 완벽하게 매핑되었습니다!")

print("\n--- 최종 데이터 미리보기 ---")
print(df_accident[['발생년월', '시군구', 'Region_Code', 'Year', 'Month']].head())

Load Success: 수도권 교통사고분석 2021~2024.csv (Enc: utf-8)
Load Success: 수도권 인구 현황 2021~2024.csv (Enc: cp949)
Load Success: 행정구역 코드.csv (Enc: utf-8)

--- 매핑 결과 ---
매핑 성공: 345180 / 355569
⚠️ 매핑 실패 지역 (2개): ['경기도 부천시' '경기도 용인시']

--- 최종 데이터 미리보기 ---
       발생년월      시군구 Region_Code  Year  Month
0  2021년 1월  경기도 오산시       31140  2021      1
1  2021년 1월  경기도 화성시       31240  2021      1
2  2021년 1월  경기도 오산시       31140  2021      1
3  2021년 1월  경기도 시흥시       31150  2021      1
4  2021년 1월  경기도 광명시       31060  2021      1


In [8]:
import pandas as pd
import os

# 1. 데이터 로드 함수 (인코딩 자동 감지)
def load_data_smart(file_path):
    encodings = ['utf-8', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            df = pd.read_csv(file_path, encoding=enc)
            print(f"Load Success: {file_path} (Enc: {enc})")
            return df
        except:
            continue
    raise ValueError(f"File Load Failed: {file_path}")

# ========================================================
# 2. 데이터 로드
# ========================================================
file_accident = "수도권 교통사고분석 2021~2024.csv"
file_pop = "수도권 인구 현황 2021~2024.csv"
file_code = "행정구역 코드.csv" 

print("--- 데이터 로드 시작 ---")
df_accident = load_data_smart(file_accident)
df_pop_raw = load_data_smart(file_pop)
df_code = load_data_smart(file_code)

# ========================================================
# 3. 행정구역 코드 정리 (5자리 코드 생성 및 딕셔너리 준비)
# ========================================================
# 시도코드(2자리) + 시군구코드(3자리) = 5자리 표준코드 생성
df_code['Sido_Str'] = df_code['시도코드'].astype(str).str.zfill(2)
df_code['Sgg_Str'] = df_code['시군구코드'].astype(str).str.zfill(3)
df_code['Region_Code'] = df_code['Sido_Str'] + df_code['Sgg_Str']

# 1차 매핑 키 생성 (예: "서울특별시 종로구")
df_code['Full_Name'] = (df_code['시도명칭'] + " " + df_code['시군구명칭']).str.strip()

# 중복 제거 (읍면동 단위 데이터가 포함되어 있으므로 시군구 단위로 유니크하게 만듦)
df_code_clean = df_code[['Full_Name', 'Region_Code']].drop_duplicates().copy()

# 1차 매핑 딕셔너리 생성
code_map = df_code_clean.set_index('Full_Name')['Region_Code'].to_dict()

# ========================================================
# 4. 사고 데이터에 적용 및 특수 처리 (핵심 수정 부분)
# ========================================================

df_accident['시군구_clean'] = df_accident['시군구'].str.strip()
temp_code_map = code_map.copy()

# 4-1. ⚠️ 매핑 실패 지역(부천시, 용인시) 특수 처리
# 교통사고 데이터에는 '경기도 부천시'만 있으나, 코드 데이터에는 '경기도 부천시 원미구' 등으로 나뉘어 있음.
missing_cities_to_fix = ['경기도 부천시', '경기도 용인시']

for city_name in missing_cities_to_fix:
    # 행정구역 코드 데이터에서 해당 City Name으로 시작하는 구 코드를 찾습니다.
    gu_codes = df_code_clean[df_code_clean['Full_Name'].str.startswith(city_name)]
    
    if not gu_codes.empty:
        # 해당 시의 '가장 낮은' 행정구역 코드(대부분 첫 번째 구)를 대표 코드로 지정하여 매핑합니다.
        representative_code = gu_codes['Region_Code'].min() 
        temp_code_map[city_name] = representative_code
        print(f"-> 특수 처리: {city_name}에 대표 코드 {representative_code} 매핑.")

# 4-2. 수정된 맵 최종 적용
df_accident['Region_Code'] = df_accident['시군구_clean'].map(temp_code_map)
df_accident.drop(columns=['시군구_clean'], inplace=True)

# 4-3. 날짜 변환 (예: '2021년 1월')
df_accident['Year'] = df_accident['발생년월'].astype(str).str[:4].astype(int)
df_accident['Month'] = df_accident['발생년월'].astype(str).str.extract(r'(\d+)월')[0].astype(int)

# ========================================================
# 5. 결과 확인
# ========================================================
print("\n--- 매핑 결과 ---")
total_rows = len(df_accident)
mapped_rows = df_accident['Region_Code'].notna().sum()
print(f"매핑 성공: {mapped_rows} / {total_rows} ({(mapped_rows/total_rows)*100:.2f}%)")

missing = df_accident[df_accident['Region_Code'].isna()]['시군구'].unique()
if len(missing) > 0:
    print(f"\n⚠️ 매핑 실패 지역 ({len(missing)}개):")
    print(missing)
else:
    print("\n✅ 모든 지역 코드가 완벽하게 매핑되었습니다! (특수 지역 처리 완료)")

print("\n--- 최종 데이터 미리보기 ---")
print(df_accident[['발생년월', '시군구', 'Region_Code', 'Year', 'Month']].head())

--- 데이터 로드 시작 ---
Load Success: 수도권 교통사고분석 2021~2024.csv (Enc: utf-8)
Load Success: 수도권 인구 현황 2021~2024.csv (Enc: cp949)
Load Success: 행정구역 코드.csv (Enc: utf-8)
-> 특수 처리: 경기도 부천시에 대표 코드 31051 매핑.
-> 특수 처리: 경기도 용인시에 대표 코드 31191 매핑.

--- 매핑 결과 ---
매핑 성공: 355569 / 355569 (100.00%)

✅ 모든 지역 코드가 완벽하게 매핑되었습니다! (특수 지역 처리 완료)

--- 최종 데이터 미리보기 ---
       발생년월      시군구 Region_Code  Year  Month
0  2021년 1월  경기도 오산시       31140  2021      1
1  2021년 1월  경기도 화성시       31240  2021      1
2  2021년 1월  경기도 오산시       31140  2021      1
3  2021년 1월  경기도 시흥시       31150  2021      1
4  2021년 1월  경기도 광명시       31060  2021      1


In [9]:
import pandas as pd
import os
import calendar
import numpy as np

# 1. 데이터 로드 함수 (인코딩 자동 감지)
def load_data_smart(file_path):
    encodings = ['utf-8', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            df = pd.read_csv(file_path, encoding=enc)
            print(f"Load Success: {file_path} (Enc: {enc})")
            return df
        except:
            continue
    raise ValueError(f"File Load Failed: {file_path}")

# ========================================================
# 2. 데이터 로드
# ========================================================
file_accident = "수도권 교통사고분석 2021~2024.csv"
file_pop = "수도권 인구 현황 2021~2024.csv"
file_code = "행정구역 코드.csv" 
file_weather = "기상청 데이터.csv"

print("--- 데이터 로드 시작 ---")
df_accident = load_data_smart(file_accident)
df_pop_raw = load_data_smart(file_pop)
df_code = load_data_smart(file_code)
df_weather_raw = load_data_smart(file_weather)


# ========================================================
# 3. 행정구역 코드 정리
# ========================================================
df_code['Sido_Str'] = df_code['시도코드'].astype(str).str.zfill(2)
df_code['Sgg_Str'] = df_code['시군구코드'].astype(str).str.zfill(3)
df_code['Region_Code'] = df_code['Sido_Str'] + df_code['Sgg_Str']

df_code['Full_Name'] = (df_code['시도명칭'] + " " + df_code['시군구명칭']).str.strip()
df_code_clean = df_code[['Full_Name', 'Region_Code']].drop_duplicates().copy()
code_map = df_code_clean.set_index('Full_Name')['Region_Code'].to_dict()

# ========================================================
# 4. 사고 데이터 전처리 및 [날짜 생성]
# ========================================================
df_accident['시군구_clean'] = df_accident['시군구'].str.strip()
temp_code_map = code_map.copy()

missing_cities_to_fix = ['경기도 부천시', '경기도 용인시']
for city_name in missing_cities_to_fix:
    gu_codes = df_code_clean[df_code_clean['Full_Name'].str.startswith(city_name)]
    if not gu_codes.empty:
        representative_code = gu_codes['Region_Code'].min() 
        temp_code_map[city_name] = representative_code
        print(f"-> 특수 처리: {city_name} -> {representative_code}")

df_accident['Region_Code'] = df_accident['시군구_clean'].map(temp_code_map)
df_accident.drop(columns=['시군구_clean'], inplace=True)

# 연도, 월 추출
df_accident['Year'] = df_accident['발생년월'].astype(str).str[:4].astype(int)
df_accident['Month'] = df_accident['발생년월'].astype(str).str.extract(r'(\d+)월')[0].astype(int)

# [핵심 로직 수정] 구분번호 순서와 주야 스위칭을 이용한 날짜(Day) 할당
print("-> 주야 스위칭 기반 날짜(Day) 할당 중...")

# 1. 구분번호 기준 정렬 (시간 순서 보장)
df_accident = df_accident.sort_values(by=['Year', 'Month', '구분번호']).reset_index(drop=True)

# 2. 월별 주야 변경 횟수(Switch Count) 계산
df_accident['Switch_Count'] = df_accident.groupby(['Year', 'Month'])['주야'].transform(lambda x: (x != x.shift()).cumsum())

# 3. 날짜(Day) 할당: (스위치 횟수 + 1) // 2
df_accident['Day'] = (df_accident['Switch_Count'] + 1) // 2

# [보정] 계산된 날짜가 해당 월의 마지막 날짜를 초과하는 경우(예: 32일), 마지막 날로 고정
def get_days_in_month(row):
    return calendar.monthrange(row['Year'], row['Month'])[1]

# 각 행별로 해당 월의 마지막 날짜(28, 30, 31 등)를 계산
df_accident['Max_Day'] = df_accident.apply(get_days_in_month, axis=1)

# 계산된 Day와 Max_Day 중 작은 값을 선택하여 날짜 초과 방지
df_accident['Day'] = np.minimum(df_accident['Day'], df_accident['Max_Day'])

# 임시 컬럼 제거
df_accident.drop(columns=['Max_Day'], inplace=True)

# [검증] 생성된 날짜 범위 확인
print("\n--- 날짜 생성 결과 확인 (일부 월) ---")
check_days = df_accident.groupby(['Year', 'Month'])['Day'].max().head()
print(check_days)
print("-> 날짜 할당 완료.")

# 시간 변수 (주야 -> 12시/0시)
def get_representative_hour(row):
    return 12 if row['주야'].strip() == '주간' else 0
df_accident['Hour'] = df_accident.apply(get_representative_hour, axis=1)


# ========================================================
# 5. 인구 데이터 전처리
# ========================================================
target_indices = [0, 1, 2, 5, 9, 12, 16, 19, 23, 26]
df_pop = df_pop_raw.iloc[:, target_indices].copy()

df_pop.columns = [
    'Sido_Nm', 'Sgg_Nm', 
    '2021_Total', '2021_Admin', 
    '2022_Total', '2022_Admin',
    '2023_Total', '2023_Admin',
    '2024_Total', '2024_Admin'
]

df_pop = df_pop[~df_pop['Sgg_Nm'].str.contains('소계|계|소재지', na=False)].reset_index(drop=True)
df_pop = df_pop.dropna(subset=['Sgg_Nm'])

df_pop_long = pd.wide_to_long(df_pop, stubnames=['Total', 'Admin'], i=['Sido_Nm', 'Sgg_Nm'], j='Year', sep='_', suffix=r'\d+').reset_index()
df_pop_long['Full_Name'] = (df_pop_long['Sido_Nm'] + " " + df_pop_long['Sgg_Nm']).str.strip()
df_pop_long['Region_Code'] = df_pop_long['Full_Name'].map(code_map)
df_pop_long = df_pop_long.dropna(subset=['Region_Code']) 
df_pop_long['Year'] = df_pop_long['Year'].astype(int)

df_pop_final = df_pop_long[['Year', 'Region_Code', 'Total', 'Admin']].copy()
df_pop_final.columns = ['Year', 'Region_Code', 'Population_Total', 'Population_Admin']

# ========================================================
# 6. 인구 데이터 병합
# ========================================================
print("-> 인구 데이터 병합 중...")
df_final = pd.merge(
    df_accident, 
    df_pop_final, 
    on=['Year', 'Region_Code'], 
    how='left'
)
df_final['Population_Feature'] = df_final['Population_Admin'] 

# ========================================================
# 7. 기상 데이터 전처리 (주간/야간 집계)
# ========================================================
print("\n--- 기상 데이터 전처리 (주간/야간 집계) ---")

df_weather = df_weather_raw.copy()
df_weather['tm_dt'] = pd.to_datetime(df_weather['tm'])
df_weather['Year'] = df_weather['tm_dt'].dt.year.astype(int)
df_weather['Month'] = df_weather['tm_dt'].dt.month.astype(int)
df_weather['Day'] = df_weather['tm_dt'].dt.day.astype(int) # 일(Day) 추가
df_weather['Hour_Raw'] = df_weather['tm_dt'].dt.hour.astype(int)

# [핵심] 기상 데이터를 사고 데이터의 'Hour(0, 12)'에 맞춰 그룹화
# 주간(12): 06시 ~ 17시 데이터 평균
# 야간(0): 18시 ~ 05시 데이터 평균
def classify_time_group(h):
    if 6 <= h < 18:
        return 12 # 주간 대표 시간
    else:
        return 0  # 야간 대표 시간

df_weather['Hour'] = df_weather['Hour_Raw'].apply(classify_time_group)

# 필요한 컬럼만 선택
cols_to_use = ['Year', 'Month', 'Day', 'Hour', 'stnId', 'ta', 'rn', 'ws']
df_weather_subset = df_weather[cols_to_use].copy()
df_weather_subset.columns = ['Year', 'Month', 'Day', 'Hour', 'stnId', 'Temperature', 'Precipitation', 'WindSpeed']

# [집계] 일별/주야별/지점별 평균 계산 (1:1 매칭을 위한 필수 과정)
print("-> 기상 데이터 집계 중 (일별 주/야 평균)...")
df_weather_agg = df_weather_subset.groupby(['Year', 'Month', 'Day', 'Hour', 'stnId']).mean().reset_index()

# --------------------------------------------------------
# 지역 코드 -> 기상 관측소 매핑
def map_region_to_stn(region_code):
    if pd.isna(region_code): return 108
    code_str = str(region_code)
    
    if code_str.startswith('11'): return 108 
    elif code_str.startswith('28'):
        if code_str == '28710': return 201
        return 112
    elif code_str.startswith('41'):
        northern = ['41480', '41250', '41650', '41800', '41630', '41280', '41150']
        eastern = ['41500', '41670', '41830', '41550', '41570', '41590']
        if code_str in northern: return 99
        elif code_str in eastern: return 203
        else: return 119
    return 108

df_final['stnId'] = df_final['Region_Code'].apply(map_region_to_stn)

print("-> 기상 데이터 병합 중 (1:1 매칭)...")
# 이제 'Day'까지 포함하여 병합하므로 정확도가 훨씬 높아짐
df_final = pd.merge(
    df_final, 
    df_weather_agg, 
    on=['Year', 'Month', 'Day', 'Hour', 'stnId'],
    how='left'
)

# 불필요한 임시 컬럼 제거
df_final.drop(columns=['Switch_Count'], inplace=True, errors='ignore')

# 결측치 확인 (기상 데이터가 없는 경우)
missing_weather = df_final['Temperature'].isna().sum()
if missing_weather > 0:
    print(f"⚠️ 기상 데이터 매칭 실패: {missing_weather}건 (전체의 {missing_weather/len(df_final)*100:.1f}%)")
    # 결측치는 평균값 등으로 간단히 채우거나 삭제 (여기서는 0으로 채움)
    df_final['Precipitation'] = df_final['Precipitation'].fillna(0)
    df_final['Temperature'] = df_final['Temperature'].fillna(method='ffill') # 이전 값으로 채움

print("-> 병합 완료.")

# ========================================================
# 8. 최종 저장
# ========================================================
output_path = 'data/수도권_교통사고_전처리_최종.csv'
output_dir = os.path.dirname(output_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_final.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 파일 저장 완료: {output_path}")
print(f"최종 데이터 크기: {df_final.shape}")
print("--- 최종 데이터 미리보기 ---")
print(df_final[['구분번호', '발생년월', 'Day', '주야', '시군구', 'Population_Feature', 'Temperature', 'Precipitation']].head())

--- 데이터 로드 시작 ---
Load Success: 수도권 교통사고분석 2021~2024.csv (Enc: utf-8)
Load Success: 수도권 인구 현황 2021~2024.csv (Enc: cp949)
Load Success: 행정구역 코드.csv (Enc: utf-8)
Load Success: 기상청 데이터.csv (Enc: utf-8)
-> 특수 처리: 경기도 부천시 -> 31051
-> 특수 처리: 경기도 용인시 -> 31191
-> 주야 스위칭 기반 날짜(Day) 할당 중...

--- 날짜 생성 결과 확인 (일부 월) ---
Year  Month
2021  1        31
      2        28
      3        31
      4        30
      5        31
Name: Day, dtype: int64
-> 날짜 할당 완료.
-> 인구 데이터 병합 중...

--- 기상 데이터 전처리 (주간/야간 집계) ---
-> 기상 데이터 집계 중 (일별 주/야 평균)...
-> 기상 데이터 병합 중 (1:1 매칭)...
-> 병합 완료.

✅ 최종 파일 저장 완료: data/수도권_교통사고_전처리_최종.csv
최종 데이터 크기: (355569, 34)
--- 최종 데이터 미리보기 ---
         구분번호      발생년월  Day  주야         시군구  Population_Feature  Temperature  \
0  2021000001  2021년 1월    1  야간   서울특별시 은평구                 NaN    -5.408333   
1  2021000002  2021년 1월    1  야간  서울특별시 영등포구                 NaN    -5.408333   
2  2021000003  2021년 1월    1  야간   서울특별시 노원구                 NaN    -5.408333   
3  2021000005  2021년 1월   

In [5]:
import pandas as pd
import numpy as np

# 1. 파일 로드
# 데이터 로드 시 인코딩 문제 해결을 위해 'utf-8' 및 'euc-kr' 인코딩을 시도합니다.
file_path = "수도권 교통사고분석 2021~2024.csv"
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='euc-kr')

# 2. '구분번호'를 기준으로 오름차순 정렬 (순차적인 변화를 확인하기 위한 필수 단계)
df_sorted = df.sort_values(by='구분번호', ascending=True)

# 3. 월별 '주야' 전환 횟수 (스위치) 계산 함수
def calculate_switches(series):
    # series != series.shift(1)은 이전 값과 다를 때 True(1)를 반환
    # 그룹의 첫 번째 행에서 발생하는 비교 오류(True)를 제외하기 위해 1을 뺌
    return (series != series.shift(1)).sum() - 1

# '발생년월'별로 그룹화하여 전환 횟수 계산
switches = df_sorted.groupby('발생년월')['주야'].apply(calculate_switches).rename('전환 횟수 (스위치)')

# 4. 사용자 정의 '사이클 횟수' 계산
# 사이클 횟수 = floor(전환 횟수 / 2)
cycles = np.floor(switches / 2).astype(int).rename('계산된 사이클 횟수')

# 결과 통합 및 출력
result_df = pd.concat([switches, cycles], axis=1)
print(result_df)

           전환 횟수 (스위치)  계산된 사이클 횟수
발생년월                              
2021년 10월           62          31
2021년 11월           60          30
2021년 12월           62          31
2021년 1월            62          31
2021년 2월            56          28
2021년 3월            62          31
2021년 4월            60          30
2021년 5월            62          31
2021년 6월            60          30
2021년 7월            62          31
2021년 8월            62          31
2021년 9월            60          30
2022년 10월           62          31
2022년 11월           60          30
2022년 12월           62          31
2022년 1월            62          31
2022년 2월            56          28
2022년 3월            62          31
2022년 4월            60          30
2022년 5월            62          31
2022년 6월            60          30
2022년 7월            62          31
2022년 8월            62          31
2022년 9월            60          30
2023년 10월           62          31
2023년 11월           60          30
2023년 12월           